# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning com Python e C++</font>
## <font color='blue'>Exportação, Importação e Serialização de Modelos de Machine Learning</font>

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Carrega o dataset
iris = load_iris()

In [3]:
# Converte para dataframe
dados = pd.DataFrame(data= np.c_[iris['data'], 
                                 iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [4]:
dados.shape

(150, 5)

In [5]:
dados.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
# Divisão em dados de treino e teste
Xtreino, Xteste, Ytreino, Yteste = train_test_split(iris.data, iris.target, test_size = 0.3, random_state = 4)

In [7]:
# Cria o modelo
modelo = LogisticRegression(C = 0.1, max_iter = 20, fit_intercept = True, solver = 'liblinear')

In [8]:
# Treina o modelo
modelo.fit(Xtreino, Ytreino)

LogisticRegression(C=0.1, max_iter=20, solver='liblinear')

## Serialização com Pickle

In [9]:
import pickle

In [10]:
# Nome do arquivo para salvar o modelo
pkl_filename = "modelos/pickle_model.pkl"

> Exportação do Modelo

In [11]:
# Abre o arquivo para escrita em formato binário e realiza o dump da memória para o disco
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelo, file)

In [12]:
# Deleta o modelo da memória
del modelo

> Importação do Modelo

In [13]:
# Carrega o modelo e coloca na memória
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [14]:
# Faz previsões com o modelo e calcula a acurácia
Ypredict = pickle_model.predict(Xteste)
acc = accuracy_score(Yteste, Ypredict)
print("Acurácia em Teste: {0:.2f} %".format(100 * acc))

Acurácia em Teste: 91.11 %


> Podemos ainda salvar e restaurar outros objetos Python

In [15]:
# Serializando outros objetos
tuple_objects = (Xtreino, Ytreino, acc)

In [16]:
# Salva em disco
pickle.dump(tuple_objects, open("modelos/tuple_model.pkl", 'wb'))

In [17]:
# Restaura os objetos
pickled_Xtreino, pickled_Ytreino, pickled_acc = pickle.load(open("modelos/tuple_model.pkl", 'rb'))

In [18]:
pickled_Xtreino[0]

array([4.3, 3. , 1.1, 0.1])

In [19]:
pickled_acc

0.9111111111111111

## Serialização com Joblib

In [20]:
import joblib

In [21]:
# Cria o modelo
modelo = LogisticRegression(C = 0.1, max_iter = 20, fit_intercept = True, solver = 'liblinear')

In [22]:
# Treina o modelo
modelo.fit(Xtreino, Ytreino)

LogisticRegression(C=0.1, max_iter=20, solver='liblinear')

In [23]:
# Nome do arquivo para salvar o modelo
joblib_file = "modelos/joblib_model.pkl"

> Exportação do Modelo

In [24]:
# Dump em disco
joblib.dump(modelo, joblib_file)

['modelos/joblib_model.pkl']

> Importação do Modelo

In [25]:
# Carrega o modelo
joblib_model = joblib.load(joblib_file)

In [26]:
# Faz previsões com o modelo e calcula a acurácia
Ypredict = joblib_model.predict(Xteste)
acc = accuracy_score(Yteste, Ypredict)
print("Acurácia em Teste: {0:.2f} %".format(100 * acc))

Acurácia em Teste: 91.11 %


## Serialização com JSON

In [27]:
import json

In [28]:
# Classe
class MyLogReg(LogisticRegression):
    
    # Construtor
    def __init__(self, C = 1.0, solver = 'liblinear', max_iter = 100, X_train = None, Y_train = None):
        LogisticRegression.__init__(self, C = C, solver = solver, max_iter = max_iter)
        self.X_train = X_train
        self.Y_train = Y_train
        
    # Método para salvar os objetos em formato JSON
    def save_json(self, filepath):
        dict_ = {}
        dict_['C'] = self.C
        dict_['max_iter'] = self.max_iter
        dict_['solver'] = self.solver
        dict_['X_train'] = self.X_train.tolist() if self.X_train is not None else 'None'
        dict_['Y_train'] = self.Y_train.tolist() if self.Y_train is not None else 'None'
        
        # Cria o JSON e salva em disco
        json_txt = json.dumps(dict_, indent = 4)
        with open(filepath, 'w') as file:
            file.write(json_txt)
    
    # Método para carregar o arquivo JSON
    def load_json(self, filepath):
        with open(filepath, 'r') as file:
            dict_ = json.load(file)
            
        self.C = dict_['C']
        self.max_iter = dict_['max_iter']
        self.solver = dict_['solver']
        self.X_train = np.asarray(dict_['X_train']) if dict_['X_train'] != 'None' else None
        self.Y_train = np.asarray(dict_['Y_train']) if dict_['Y_train'] != 'None' else None

In [29]:
# Caminho e nome do arquivo
filepath = "modelos/mylogreg.json"

In [30]:
# Cria o modelo
modelo = MyLogReg(X_train = Xtreino, Y_train = Ytreino)

In [31]:
# Treina o modelo
mylogreg = modelo.fit(Xtreino, Ytreino)

In [32]:
# Faz previsões com o modelo e calcula a acurácia
Ypredict = mylogreg.predict(Xteste)
acc = accuracy_score(Yteste, Ypredict)
print("Acurácia em Teste: {0:.2f} %".format(100 * acc))

Acurácia em Teste: 93.33 %


In [33]:
# Salva o modelo em disco no formato JSON
mylogreg.save_json(filepath)

In [34]:
# Deleta o modelo
del mylogreg

In [35]:
# Carrega o modelo
json_mylogreg = MyLogReg()
json_mylogreg.load_json(filepath)

In [36]:
# Faz previsões com o modelo e calcula a acurácia
Ypredict = json_mylogreg.predict(Xteste)
acc = accuracy_score(Yteste, Ypredict)
print("Acurácia em Teste: {0:.2f} %".format(100 * acc))

NotFittedError: This MyLogReg instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [37]:
# Treina o modelo
json_mylogreg.fit(Xtreino, Ytreino)

MyLogReg(X_train=array([[4.3, 3. , 1.1, 0.1],
       [5.7, 4.4, 1.5, 0.4],
       [5.9, 3. , 4.2, 1.5],
       [6.1, 3. , 4.6, 1.4],
       [6.5, 3. , 5.5, 1.8],
       [5.2, 3.5, 1.5, 0.2],
       [5.6, 2.5, 3.9, 1.1],
       [7.7, 2.6, 6.9, 2.3],
       [6.3, 3.4, 5.6, 2.4],
       [6.2, 2.9, 4.3, 1.3],
       [5.7, 2.9, 4.2, 1.3],
       [5. , 3.5, 1.6, 0.6],
       [5.6, 2.9, 3.6, 1.3],
       [6. , 2.2, 5. , 1.5],
       [5.5, 2.6, 4.4, 1.2],
       [4.6, 3.4, 1.4, 0.3],
       [5.6, 3. , 4.1, 1.3],
       [5.1, 3.4, 1.5, 0.2],
       [6.4, 2.9, 4...
       [6.6, 2.9, 4.6, 1.3],
       [6.4, 3.1, 5.5, 1.8],
       [7. , 3.2, 4.7, 1.4],
       [6.3, 2.3, 4.4, 1.3],
       [6.5, 3. , 5.8, 2.2],
       [7.2, 3. , 5.8, 1.6],
       [7.7, 2.8, 6.7, 2. ]]),
         Y_train=array([0, 0, 1, 1, 2, 0, 1, 2, 2, 1, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2,
       1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 2, 1, 0, 2, 1, 2, 0, 2, 1, 2, 1, 2,
       1, 1, 2, 1, 1, 2, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 2, 2, 1, 

In [38]:
# Faz previsões com o modelo e calcula a acurácia
Ypredict = json_mylogreg.predict(Xteste)
acc = accuracy_score(Yteste, Ypredict)
print("Acurácia em Teste: {0:.2f} %".format(100 * acc))

Acurácia em Teste: 93.33 %


Uma vez que a serialização de dados usando JSON realmente salva o objeto em um formato de string, em vez de fluxo de bytes, o arquivo 'mylogreg.json' pode ser aberto e modificado com um editor de texto. 

Embora esta abordagem seja conveniente para o desenvolvedor, é menos segura porque um intruso pode visualizar e alterar o conteúdo do arquivo JSON. Além disso, esta abordagem é mais adequada para objetos com pequeno número de variáveis, como os modelos scikit-learn, porque qualquer adição de novas variáveis requer mudanças nos métodos de salvar e restaurar.

In [39]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [40]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

json  : 2.0.9
joblib: 1.0.1
numpy : 1.21.2
pandas: 1.3.3



# Fim